In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

Читаем входные файлы с данными

In [3]:
transactions = pd.read_csv('data/transactions.csv')
customers_gender = pd.read_csv('data/customers_gender_train.csv')

In [4]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [16]:
X = transactions.groupby('customer_id').apply(lambda x: x[['tr_type']].unstack().value_counts()) \
                  .unstack() \
                 .fillna(0)

Для каждого клиента подсчитываем различные mcc_code

In [26]:
X = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

In [28]:
X.shape

(15000, 184)

In [18]:
customers_gender = customers_gender.set_index('customer_id')

In [19]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [20]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

In [21]:
X_train.head()

,1000,1010,1030,1100,1110,1200,1210,1310,1410,1510,...,7040,7041,7044,7070,7071,7074,7075,8100,8145,8146
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,0,42,90,0,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22899,0,6,47,0,27,1,0,0,0,0,...,0,0,0,9,3,0,1,0,0,0
27914,0,0,58,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28753,0,89,25,3,65,0,0,0,0,0,...,1,0,0,14,0,0,0,0,0,0
31385,0,75,125,0,75,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Обучаемся на всех данных

In [22]:
import xgboost as xgb
from sklearn.cross_validation import train_test_split
Xtr, Xval, ytr, yval = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [23]:
params = {
    'max_depth': 6,
    'eta': 0.025,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['auc'],
    'objective': 'binary:logistic',
    'colsample_bytree': 0.9,
    'min_child_weight': 0,
    #'scale_pos_weight':(1-train.Label.mean())/train.Label.mean(),
    'seed':1337
}

In [24]:
dtr = xgb.DMatrix(Xtr, label=ytr)
dval = xgb.DMatrix(Xval, label=yval)
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [25]:
model = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=10)

[0]	train-auc:0.633928	eval-auc:0.603568
[10]	train-auc:0.699584	eval-auc:0.619718
[20]	train-auc:0.710837	eval-auc:0.62152
[30]	train-auc:0.720957	eval-auc:0.627249
[40]	train-auc:0.729549	eval-auc:0.628929
[50]	train-auc:0.739267	eval-auc:0.632356
[60]	train-auc:0.749146	eval-auc:0.637945
[70]	train-auc:0.754515	eval-auc:0.638501
[80]	train-auc:0.761423	eval-auc:0.640597
[90]	train-auc:0.76824	eval-auc:0.642113
[100]	train-auc:0.77321	eval-auc:0.643467
[110]	train-auc:0.779219	eval-auc:0.645564
[120]	train-auc:0.784826	eval-auc:0.646844
[130]	train-auc:0.790184	eval-auc:0.648763
[140]	train-auc:0.794797	eval-auc:0.650692
[150]	train-auc:0.800104	eval-auc:0.651545
[160]	train-auc:0.804414	eval-auc:0.652142
[170]	train-auc:0.807882	eval-auc:0.653232
[180]	train-auc:0.810775	eval-auc:0.654508
[190]	train-auc:0.815183	eval-auc:0.654851
[200]	train-auc:0.818905	eval-auc:0.655867
[210]	train-auc:0.822112	eval-auc:0.656
[220]	train-auc:0.825586	eval-auc:0.655534
[230]	train-auc:0.828758	eva

In [7]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train.values[:, 0])

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=13, subsample=1.0, verbose=0,
              warm_start=False)

Предсказываем пол для тестовых данных и создаём файл с ответом

In [10]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [11]:
result.to_csv('baseline_a.csv', index=False)